# `concurrent.futures` Executor

In [1]:
%run -m literary.notebook

import time

In [2]:
import asyncio
import weakref
from concurrent import futures

from .executor import Executor
from .futures import chain_cancellation, create_future
from .wrap import AttributeHandleWrapper

The `ConcurrentFuturesExecutor` accepts a `futures.Executor` object:

In [3]:
class ConcurrentFuturesExecutor(Executor):
    def __init__(self, executor: futures.Executor):
        self._executor = pool
        self._wrapper = AttributeHandleWrapper()

Because `futures.Executor` instances act as context managers, we should implement the same interface.

In [4]:
@patch(ConcurrentFuturesExecutor)
def __enter__(self):
    self._executor.__enter__()
    return self

In [5]:
@patch(ConcurrentFuturesExecutor)
def __exit__(self, exc_type, exc_val, exc_tb):
    self._executor.__exit__(exc_type, exc_val, exc_tb)
    return

To submit a task, we launch a `_submit` task which dispatches to the `futures.Executor`, and then chain the task with our handle.

In [6]:
@patch(ConcurrentFuturesExecutor)
def submit(self, func, /, *args, **kwargs) -> asyncio.Future:
    handle = create_future()

    # Because the unwrap stage actually needs to wait for results,
    # we create a task to do this work
    task = asyncio.create_task(self._submit(func, handle, *args, **kwargs))
    # Cancelling the handle should cancel the task
    chain_cancellation(handle, task)
    return handle

In this `_submit` method, we invoke the `futures.Executor.submit` method, and wrap the result in an `asyncio.Future` handle. This future holds the status of the running task. 

In [7]:
@patch(ConcurrentFuturesExecutor)
async def _submit(self, func, handle: asyncio.Future, /, *args, **kwargs):
    args, kwargs = await self._process_args(args, kwargs)
    # Launch task into pool
    cf_fut = self._executor.submit(func, *args, **kwargs)

    # Ensure partial two-way sync between CF and asyncio
    self._chain_futures(handle, cf_fut)

    # Wrap CF future in handle
    self._wrapper.wrap(handle, cf_fut)

The `futures.Executor.submit` method cannot make use of `futures.Future` arguments. Instead, we have to resolve them to their computed values by waiting on them first:

In [8]:
@patch(ConcurrentFuturesExecutor)
async def _process_args(self, args, kwargs):
    # Unwrap any wrapped handles
    args = [await self._unwrap_and_wait_maybe(x) for x in args]
    kwargs = {k: await self._unwrap_and_wait_maybe(v) for k, v in kwargs.items()}
    return args, kwargs

Here, we unwrap the `asyncio.Future` handles in the arguments, and await their results so that can be passed to `futures.Executor.submit`.

In [9]:
@patch(ConcurrentFuturesExecutor)
async def _unwrap_and_wait_maybe(self, obj):
    # Ensure is wrapped
    try:
        self._wrapper.unwrap(obj)
    except AttributeError:
        return obj

    return await self.retrieve(obj)

In order to connect the handle with the `futures.Future` result, we implement a routine to chain these with `asyncio.Future` objects:

In [10]:
@patch(ConcurrentFuturesExecutor)
def _chain_futures(self, handle: asyncio.Future, cf_fut: futures.Future):
    def on_cf_fut_done_threadsafe(cf_fut, handle_ref=weakref.ref(handle)):
        if not (fut := handle_ref()):
            return

        if fut.cancelled():
            return

        if cf_fut.cancelled():
            fut.cancel()
        elif cf_fut.done():
            fut.set_result(True)
        else:
            fut.set_exception(cf_fut.exception)

    loop = asyncio.get_running_loop()

    @cf_fut.add_done_callback
    def on_cf_fut_done(cf_fut):
        loop.call_soon_threadsafe(on_cf_fut_done_threadsafe, cf_fut)

    @handle.add_done_callback
    def on_fut_done(fut):
        if fut.cancelled():
            cf_fut.cancel()

Finally, we implement a method to unwrap and return the result object for the `futures.Future`:

In [11]:
@patch(ConcurrentFuturesExecutor)
async def retrieve(self, handle: asyncio.Future):
    await handle
    return self._wrapper.unwrap(handle).result()

To demonstrate this, we can create a thread pool executor:

In [12]:
pool = futures.ProcessPoolExecutor()

Using this pool we can create an exectutor:

In [13]:
executor = ConcurrentFuturesExecutor(pool)

To do some work, let's implement a sleep function that returns the delay

In [14]:
def slow_function(timeout):
    time.sleep(timeout)
    return timeout

Now we can chain a few of these tasks together:

In [15]:
a = executor.submit(
    slow_function,
    2,
)
b = executor.submit(slow_function, 5)
c = executor.submit(int.__add__, a, b)

We can wait for the result without retrieving its value:

In [16]:
await c

True

And when we're ready for the value, we invoke `executor.retrieve`.

In [17]:
executor._wrapper.unwrap(c).result()

7

In [18]:
assert await executor.retrieve(c) == 7